In [2]:
import csv
import json
import matplotlib.pyplot as plt
import requests
import pandas as pd
from pprint import pprint
import numpy as np

response = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CR2016Geo.json?lang=E&geos=PR&topic=2&&notes=0')

b = json.loads(response.text[2:])
#pprint(b['DATA'])

prov = []
dguid=[]
for i in range((len(b['DATA']))):
    prov.append(b['DATA'][i][2])
    dguid.append(b['DATA'][i][0])

province_dguid = pd.DataFrame({'Province':prov,'dguid':dguid})
province_dguid

aboriginal = pd.DataFrame()
for id in province_dguid['dguid']:
    response2 = requests.get('https://www12.statcan.gc.ca/rest/census-recensement/CPR2016.json?lang=E&dguid='+id+'&topic=1&&notes=0')
    b2 = json.loads(response2.text[2:])
    #pprint(b2['DATA'])
    aboriginal = aboriginal.append(b2['DATA'])
    
#aboriginal

In [4]:
#abo = aboriginal.filter(items=['Total - Aboriginal identity for the population in private households - 25% sample data'])
aboriginal_t = aboriginal[aboriginal[10].str.contains('Total - Aboriginal identity for the population in private households - 25% sample data')]
aboriginal_a = aboriginal[aboriginal[10].str.contains('  Aboriginal identity')]
aboriginal_n = aboriginal[aboriginal[10].str.contains('  Non-Aboriginal identity')]
abo1 = pd.DataFrame({'Provinces':aboriginal_t[1],'Total':aboriginal_t[13]}).set_index('Provinces')
abo2 = pd.DataFrame({'Provinces':aboriginal_a[1],'Aboriginal':aboriginal_a[13]}).set_index('Provinces')
abo3 = pd.DataFrame({'Provinces':aboriginal_n[1],'Non-aboriginal':aboriginal_n[13]}).set_index('Provinces')
abo = pd.merge(abo1,abo2, on='Provinces')
abo = pd.merge(abo,abo3, on='Provinces')
abo['Aboriginal %'] = 100 * (abo['Aboriginal']/abo['Total'])
abo['Non-aboriginal %'] = 100 * (abo['Non-aboriginal']/abo['Total'])
abo

,Total,Aboriginal,Non-aboriginal,Aboriginal %,Non-aboriginal %
Provinces,,,,,
Canada,34460065.0,1673785.0,32786280.0,4.857173,95.142827
Newfoundland and Labrador,512255.0,45730.0,466525.0,8.927194,91.072806
Prince Edward Island,139685.0,2735.0,136945.0,1.957977,98.038444
Nova Scotia,908340.0,51495.0,856850.0,5.669133,94.331418
New Brunswick,730710.0,29385.0,701325.0,4.021431,95.978569
Quebec,7965450.0,182885.0,7782560.0,2.295978,97.703959
Ontario,13242160.0,374395.0,12867765.0,2.827296,97.172704
Manitoba,1240700.0,223310.0,1017385.0,17.998710,82.000887
Saskatchewan,1070555.0,175020.0,895535.0,16.348530,83.651470


In [7]:
aboriginal.to_csv("Aboriginal Raw Data.csv",encoding="utf-8", index=True, header=True)
abo.to_csv("Aboriginal Population Per Province.csv",encoding="utf-8", index=True, header=True)